# OMMX Python SDK 1.7.0

[![Static Badge](https://img.shields.io/badge/GitHub_Release-Python_SDK_1.7.0-blue?logo=github)](https://github.com/Jij-Inc/ommx/releases/tag/python-1.7.0)

Summary
-------
- [英語のJupyter Book](https://jij-inc.github.io/ommx/en/introduction.html)
- QPLIBフォーマットのパーサーが追加されました
- `ommx.v1.SampleSet` と `ommx.v1.ParametricInstance` にいくつかAPIが追加されました。またOMMX Artifactとの連携機能が追加されました。
- `ommx.v1.Solution` と `ommx.v1.SampleSet` にある `feasible` の挙動が変更されました
  - Python SDK 1.6.0で導入された `removed_constraints` の扱いが変更されています。1.6.0では `feasible` は `removed_constraints` を無視していましたが、1.7.0では `removed_constraints` を考慮するようになりました。
  - 合わせて明示的に `removed_constraints` を無視する `feasible_relaxed` と考慮する `feasible_unrelaxed` を導入しました。`feasible` は `feasible_unrelaxed` のエイリアスになっています。